1. Скачать оригинальный датасет
2. Привести значения тестового датасета к оригинальным значениям изначального датасета.
3. Обучить мультиклассовую классификацию и посмотреть на ошибки модели
4. Объеденить рядомстоящие классы с наименьшей точностью в группы
5. Обучить отдельные модели на таких группах

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool
from tqdm import tqdm
import optuna

In [2]:
from utility import make_BMI, BMI_to_str

Содержание опроса оригинального опроса:

Original dataset

Gender - Female/Male

age - Numeric value

height - Numeric value in meters

weight - Numeric value in kilograms

Has a family member suffered or suffers from overweight - Yes/No

Do you eat high caloric food frequently - Yes/No

Do you usually eat vegetables in your meals - Never/Sometimes/Always

How many main meals do you have daily - Between 1 y 2/Three/More than three

Do you eat any food between meals? No/Sometimes/Frequently/Always

Do you smoke? Yes/No

How much water do you drink daily? Less than a liter/Between 1 and 2 L/More than 2 L

Do you monitor the calories you eat daily - Yes/No

How often do you have physical activity? I do not have/1 or 2 days/2 or 4 days/4 or 5 days

How much time do you use technological devices such as cell phone, videogames, television, computer and others -
0–2 hours/3–5 hours/More than 5 hours

how often do you drink alcohol? - I do not drink/Sometimes/Frequently/Always

Which transportation do you usually use? Automobile/Motorbike/Bike/Public Transportation/Walking

In [3]:
data = pd.read_csv('train.csv', index_col='id')
original = pd.read_csv('ObesityDataSet.csv')

original.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [15]:
original['BMI'] = make_BMI(original).round(1)
original['BMIsign'] = original['BMI'].apply(BMI_to_str)
original.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,BMI,BMIsign
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight,24.4,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight,24.2,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight,23.8,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I,26.9,Overweight
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II,28.3,Overweight


In [16]:
original[(original['NObeyesdad'] != original['BMIsign']) & (original['BMIsign'] != 'Overweight')]

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,BMI,BMIsign
498,Female,25.196214,1.686306,104.572712,yes,yes,3.000000,3.000000,Sometimes,no,1.152736,no,0.319156,1.000000,Sometimes,Public_Transportation,Obesity_Type_III,36.8,Obesity_Type_II
505,Female,26.000000,1.629191,104.826776,yes,yes,3.000000,3.000000,Sometimes,no,2.654702,no,0.000000,0.555468,Sometimes,Public_Transportation,Obesity_Type_III,39.5,Obesity_Type_II
524,Female,20.225396,1.550648,44.641796,no,yes,3.000000,2.857787,Frequently,no,1.000000,no,0.754646,0.000000,Sometimes,Public_Transportation,Insufficient_Weight,18.6,Normal_Weight
532,Male,22.851834,1.854592,63.611109,yes,yes,3.000000,3.691226,Sometimes,no,2.650989,no,1.228136,0.832400,Sometimes,Automobile,Insufficient_Weight,18.5,Normal_Weight
549,Female,29.970445,1.610863,49.516027,yes,yes,2.059138,3.904858,Frequently,no,2.000000,no,0.821977,0.000000,no,Public_Transportation,Insufficient_Weight,19.1,Normal_Weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,Female,25.289428,1.686033,104.772164,yes,yes,3.000000,3.000000,Sometimes,no,1.299194,no,0.234303,0.946888,Sometimes,Public_Transportation,Obesity_Type_III,36.9,Obesity_Type_II
2044,Female,25.561868,1.675185,110.621723,yes,yes,3.000000,3.000000,Sometimes,no,1.495830,no,0.109327,0.384129,Sometimes,Public_Transportation,Obesity_Type_III,39.4,Obesity_Type_II
2068,Female,24.196367,1.697421,114.482386,yes,yes,3.000000,3.000000,Sometimes,no,2.909675,no,0.360908,0.573887,Sometimes,Public_Transportation,Obesity_Type_III,39.7,Obesity_Type_II
2096,Female,25.955014,1.626449,104.879602,yes,yes,3.000000,3.000000,Sometimes,no,2.094901,no,0.070890,0.599441,Sometimes,Public_Transportation,Obesity_Type_III,39.6,Obesity_Type_II


In [38]:
original.shape

(2111, 19)

In [36]:
data['BMI'] = make_BMI(data)
data['BMIsign'] = data['BMI'].apply(BMI_to_str)

X_train, X_valid, y_train, y_valid = train_test_split(data[['Age', 'BMI', 'BMIsign']],
                                                      data['NObeyesdad'],
                                                      test_size=0.20,
                                                      random_state=3,
                                                      shuffle=True)

clf = CatBoostClassifier()
clf.fit(X_train, y_train, cat_features=['BMIsign'], eval_set=(X_valid, y_valid), silent=True, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [37]:
clf.score(X_valid, y_valid)

0.8489884393063584